1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [1]:
# !pip install bs4

In [2]:
# импорт библиотек
import requests
from pprint import pprint
from bs4 import BeautifulSoup as bs
import pandas as pd
from pymongo import MongoClient

In [3]:
# Подключаем базу
client = MongoClient('127.0.0.1', 27017)
db = client['Jobs']
hh_vacancy = db.hh_vacancy

In [4]:
url_hh = 'https://hh.ru/search/vacancy'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 YaBrowser/23.1.2.931 Yowser/2.5 Safari/537.36'}
resp = requests.get(url_hh, headers=headers)

In [5]:
# Проверяем есть ли уже такая запись в базе
def check(s):
    for doc in hh_vacancy.find({'link': s}):
        return True

In [6]:

# Функции преобразования данных
def salary_prep(salary):
    '''Преобразование данных по заработной плате'''
    try:
        salary = salary.text
    except:
        salary = None
    if salary == None:
        salary_min = None
        salary_max = None
        salary_currency = None
    else: 
        salary = salary.replace("\u202f", "") 
        salary = salary.replace("\xa0", "")
        salary_currency = None
        salary_list = salary.split(' ')
        if salary_list[0] == 'от':
            salary_min = int(salary_list[1])
            salary_max = None
            salary_currency = salary_list[-1]
        elif salary_list[0] == 'до':
            salary_min = None
            salary_max = int(salary_list[1])
            salary_currency = salary_list[-1]
        elif salary_list[1] == '-':
            salary_min = int(salary_list[0])
            salary_max = int(salary_list[2])
            salary_currency = salary_list[-1]    
        else:
            salary_min = int(salary_list[0])
            salary_max = int(salary_list[0])
            salary_currency = salary_list[-1]
#     salary_min = float(salary_min)
#     salary_max = float(salary_max)
   

    
    salary_list = [salary_min, salary_max, salary_currency]    
        
    return salary_list

# Функция замены закодированных пробелов в тексте
def text_prep(text):
    text = text.replace("\xa0", " ")
    text_list = text.split(',')
    if text_list[0] == None:
        text_list[0] = "None"
    return text_list[0]

In [7]:
# Функция парсинга HH
def get_conten_hh(resp): 
    soup = bs(resp.text, 'html.parser')
    items = soup.find_all('div', {'class':'vacancy-serp-item__layout'})
    hh_vacancy = []

    for item in items:
        salary = item.find('span', {'class':'bloko-header-section-3'})
        hh_vacancy.append(
            {
                'site': 'HeadHanter', # Название сайта
                'title': item.find('a', {'class':'serp-item__title'}).get_text(), # Название вакансии
                'link': item.find('a', {'class':'serp-item__title'}).get('href'), # Ссылка на вакансию
                'salary_min': salary_prep(salary)[0], # Зарплата минимальная
                'salary_max': salary_prep(salary)[1], # Зарплата максимальная
                'salary_currency': salary_prep(salary)[2], # Валюта зарплаты
                'city': text_prep(item.find('div', {'data-qa':'vacancy-serp__vacancy-address'}).get_text()), # Город
                'organization': text_prep(item.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}).get_text()), # Наименование компании
                }
            )

    return hh_vacancy

In [14]:
# Функция парсинга HH и отбоара новых вакансий
def get_new_conten_hh(resp): 
    soup = bs(resp.text, 'html.parser')
    items = soup.find_all('div', {'class':'vacancy-serp-item__layout'})
    hh_new_vacancy = []

    for item in items:
        salary = item.find('span', {'class':'bloko-header-section-3'})
        salsry_link = item.find('a', {'class':'serp-item__title'}).get('href')
        if check(salsry_link) is True:
            print(salsry_link)
            continue
        else:
#             Добавляем запись в базу
            vacancy_data ={
                    'site': 'HeadHanter', # Название сайта
                    'title': item.find('a', {'class':'serp-item__title'}).get_text(), # Название вакансии
                    'link': salsry_link, # Ссылка на вакансию
                    'salary_min': salary_prep(salary)[0], # Зарплата минимальная
                    'salary_max': salary_prep(salary)[1], # Зарплата максимальная
                    'salary_currency': salary_prep(salary)[2], # Валюта зарплаты
                    'city': text_prep(item.find('div', {'data-qa':'vacancy-serp__vacancy-address'}).get_text()), # Город
                    'organization': text_prep(item.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}).get_text()), # Наименование компании
                    }
            hh_new_vacancy.append(vacancy_data)
            hh_vacancy.insert_one(vacancy_data)

    return hh_new_vacancy
# get_new_conten_hh(resp)

In [22]:
# Функция постраничного парсинга
def parser_new_hh():
    post = str(input('Введите название вакансии для парсинга: '))
#     pages = int(input('Количество страниц для парсинга: '))
    html = resp
    print(f'количество элементов в БД до запуска парсера{len(list(hh_vacancy.find()))}')
    if html.status_code == 200:
        new_vacancy = []
        page = 0
        while True:          
#         for page in range(1, pages + 1):
            print(f'Парсим страницу {page+1}')
            html = requests.get(url_hh, headers=headers, params={'text': post, 'page': page, 'items_on_page': 20, 'area': 1463})
            new_vacancy.extend(get_new_conten_hh(html))
            dom = bs(html.text, 'html.parser')
            link_next_page = dom.find('a', {'data-qa':'pager-next'})
            page +=1
            if not dom.find('a', {'data-qa':'pager-next'}): #or not html.ok
                print("Парсинг завершен")
                break
       
            
#         hh_vacancy.insertMany(new_vacancy)

        hh_result = pd.DataFrame(new_vacancy)
    else:
        hh_result = html.status_code
        print("Парсинг завершен")
    print(f'количество элеменотовв БД после запуска парсера {len(list(hh_vacancy.find()))}, добавлено {len(new_vacancy)} элементов')    
#     hh_result.to_csv (r'vacancy.csv', index= False )
   
    return hh_result



In [23]:
parser_new_hh()

Введите название вакансии для парсинга: Водитель
количество элементов в БД до запуска парсера36
Парсим страницу 1
Парсим страницу 2
Парсим страницу 3
Парсинг завершен
количество элеменотовв БД после запуска парсера 95, добавлено 59 элементов


,site,title,link,salary_min,salary_max,salary_currency,city,organization,_id
0,HeadHanter,Водитель,https://lyubertsy.hh.ru/vacancy/77207729?from=...,37000.0,37000.0,руб.,Нальчик,НТ,63fd1fe05df4687e34b1cf49
1,HeadHanter,Водитель-экспедитор,https://lyubertsy.hh.ru/vacancy/77243009?from=...,70000.0,NaN,руб.,Прохладный,ООО ЭКСПОРТ ГРЕЙН,63fd1fe05df4687e34b1cf4a
2,HeadHanter,Водитель-экспедитор,https://lyubertsy.hh.ru/vacancy/77207747?from=...,37000.0,37000.0,руб.,Нальчик,НТ,63fd1fe05df4687e34b1cf4b
3,HeadHanter,Личный водитель-Помощник,https://lyubertsy.hh.ru/vacancy/74390845?from=...,30000.0,30000.0,руб.,Нальчик,ООО Сады Эльбруса,63fd1fe05df4687e34b1cf4c
4,HeadHanter,"Диспетчер, удаленно (в Я.Такси)",https://lyubertsy.hh.ru/vacancy/76936367?from=...,24000.0,NaN,руб.,Нальчик,Яндекс,63fd1fe05df4687e34b1cf4d
5,HeadHanter,"Диспетчер, удаленно (в Я.Такси)",https://lyubertsy.hh.ru/vacancy/76935681?from=...,24000.0,NaN,руб.,Прохладный,Яндекс,63fd1fe05df4687e34b1cf4e
6,HeadHanter,"Диспетчер, удаленно (в Я.Такси)",https://lyubertsy.hh.ru/vacancy/76935679?from=...,24000.0,NaN,руб.,Тырныауз,Яндекс,63fd1fe05df4687e34b1cf4f
7,HeadHanter,Менеджер поддержки Яндекс.Go (удаленно),https://lyubertsy.hh.ru/vacancy/77432255?from=...,24000.0,NaN,руб.,Нальчик,Яндекс,63fd1fe05df4687e34b1cf50
8,HeadHanter,Менеджер поддержки Яндекс.Go (удаленно),https://lyubertsy.hh.ru/vacancy/77431496?from=...,24000.0,NaN,руб.,Прохладный,Яндекс,63fd1fe05df4687e34b1cf51
9,HeadHanter,Менеджер поддержки Яндекс.Go (удаленно),https://lyubertsy.hh.ru/vacancy/77431492?from=...,24000.0,NaN,руб.,Тырныауз,Яндекс,63fd1fe05df4687e34b1cf52


2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

In [25]:
# Проверяем есть ли null
def check(s):
    if s:
        return s
    else:
        return int('0')


client = MongoClient('127.0.0.1', 27017)
db = client['Jobs']
hh_vacancy = db.hh_vacancy

salary_info = int(input('Введите желаемую зарплату в рублях: '))

for doc in hh_vacancy.find({'$or': [{'salary_currency': 'руб.'},
                                    {'salary_currency': 'USD'},
                                    {'salary_currency': 'EUR'},
                                    ]
                            },
                           ):
    if doc['salary_currency'] == 'руб.':
        if check(doc['salary_min']) > salary_info or check(doc['salary_max']) > salary_info:
            pprint(doc)
    elif doc['salary_currency'] == 'USD':
        if (check(doc['salary_min']) * 75) > salary_info or (check(doc['salary_max']) * 75) > salary_info:
            pprint(doc)
    elif doc['salary_currency'] == 'EUR':
        if (check(doc['salary_min']) * 80) > salary_info or (check(doc['salary_max']) * 85) > salary_info:
            pprint(doc)
    else:
        continue

Введите желаемую зарплату: 120000
{'_id': ObjectId('63fd1bd45df4687e34b1cf32'),
 'city': 'Черкесск',
 'link': 'https://lyubertsy.hh.ru/vacancy/76564206?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'organization': 'АО Газстройпром',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 130000,
 'site': 'HeadHanter',
 'title': 'Водитель автомобиля (трубоплетевоз)'}
{'_id': ObjectId('63fd1bd45df4687e34b1cf33'),
 'city': 'Псыж',
 'link': 'https://lyubertsy.hh.ru/vacancy/76562244?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'organization': 'АО Газстройпром',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 130000,
 'site': 'HeadHanter',
 'title': 'Водитель автомобиля (самосвал)'}
{'_id': ObjectId('63fd1bd45df4687e34b1cf34'),
 'city': 'Черкесск',
 'link': 'https://lyubertsy.hh.ru/vacancy/76560166?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'organization'